In [ ]:
import copy

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from tqdm.auto import tqdm
from scipy.stats import linregress

In [ ]:
f = FAIR()

In [ ]:
f.define_time(1750, 2151, 1)

In [ ]:
scen_df = pd.read_csv('../data/c1_emissions_scenarios.csv')

In [ ]:
scen_labels = pd.unique(list(zip(scen_df.Model, scen_df.Scenario)))

In [ ]:
scenarios = [f'{scen[0]}___{scen[1]}' for scen in scen_labels]

In [ ]:
# do one at a time
#scenarios = [scenarios[0]]
f.define_scenarios(scenarios)
fair_params_df = pd.read_csv('../data/calibrated_constrained_parameters.csv', index_col=0)

In [ ]:
f.define_configs(list(fair_params_df.index))

In [ ]:
species = ['CO2', 'CH4', 'N2O']
properties = {
    "CO2": {
        'type': 'co2',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "CH4": {
        'type': 'ch4',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "N2O": {
        'type': 'n2o',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
# declare species and properties
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
hist_df = pd.read_csv('../data/historical_emissions.csv')

In [ ]:
hist_df
pd.unique(scen_df['Variable'])

In [ ]:
fair_params_df['co2_concentration_1750']

In [ ]:
for scenario in scenarios:
    f.emissions.loc[
        dict(scenario=scenario, specie='CO2', timepoints=np.arange(1750.5, 2015))
    ] = hist_df.loc[hist_df['Variable'].str.startswith("Emissions|CO2|"), '1750':'2014'].values.sum(axis=0, keepdims=True).T * 1/1000
    f.emissions.loc[dict(scenario=scenario, specie='CH4')] = 0
    f.emissions.loc[dict(scenario=scenario, specie='N2O')] = 0
    f.concentration.loc[dict(scenario=scenario, specie='CH4')] = 729.2
    f.concentration.loc[dict(scenario=scenario, specie='N2O')] = 270.1
    
    model, scen = scenario.split('___')
    f.emissions.loc[
        dict(scenario=scenario, specie='CO2', timepoints=np.arange(2015.5, 2101))
    ] = scen_df.loc[
        (scen_df['Variable'].str.startswith("AR6 climate diagnostics|Infilled|Emissions|CO2|")) &
        (scen_df['Scenario']==scen) &
        (scen_df['Model']==model),
        '2015':'2100'
    ].values.sum(axis=0, keepdims=True).T * 1/1000
    
    f.emissions.loc[
        dict(scenario=scenario, specie='CO2', timepoints=np.arange(2100.5, 2151))
    ] = f.emissions.loc[
        dict(scenario=scenario, specie='CO2', timepoints=np.arange(2100.5, 2101.5))
    ].values * np.linspace(1, 0, 51)[:, None]

In [ ]:
pl.plot(f.emissions.loc[
        dict(scenario=scenario, specie='CO2', config=fair_params_df.index[0])
])

In [ ]:
# Get default species configs
f.fill_species_configs()

# Climate response
fill(f.climate_configs['ocean_heat_capacity'], fair_params_df.loc[:,'c1':'c3'])
fill(f.climate_configs['ocean_heat_transfer'], fair_params_df.loc[:,'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[:,'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], fair_params_df.loc[:,'gamma'])
fill(f.climate_configs['stochastic_run'], False)
fill(f.climate_configs["forcing_4co2"], fair_params_df.loc[:,"F_4xCO2"])

# carbon cycle
fill(f.species_configs['iirf_0'], fair_params_df.loc[:, 'r0'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_airborne'], fair_params_df.loc[:, 'rA'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_uptake'], fair_params_df.loc[:, 'rU'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_temperature'], fair_params_df.loc[:, 'rT'].values.squeeze(), specie='CO2')

# Scale CO2 forcing based on its 4xCO2 calibration
fill(f.species_configs["forcing_scale"], fair_params_df["scale CO2"].values.squeeze(), specie='CO2')

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs['baseline_concentration'], 
    fair_params_df.loc[:, 'co2_concentration_1750'].values.squeeze(),
    specie='CO2'
)

fill(f.species_configs['forcing_reference_concentration'], 729.2, specie='CH4')
fill(f.species_configs['forcing_reference_concentration'], 270.1, specie='N2O')

In [ ]:
f.species_configs['forcing_reference_concentration']

In [ ]:
# set initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

In [ ]:
f.run()

In [ ]:
pl.plot(f.temperature[:,96,:,0]);

In [ ]:
pl.plot(f.emissions[:, 96, 0, 0])

In [ ]:
pl.plot(f.concentration[:,96,:,0]);

## Find year of net zero CO2

In [ ]:
first_negative = np.array([f.emissions[:,:,0,0]<0]).argmax(axis=1).squeeze()
first_negative

In [ ]:
#f.emissions[-5:,9,0,0]
# need to exclude scenarios 2, 7 and 9
f.cumulative_emissions[first_negative, :, 0, 0]

In [ ]:
netzero_to_2150_cumulative_emissions = np.zeros(97)
for iscen in tqdm(range(97)):
    netzero_to_2150_cumulative_emissions[iscen] = (f.cumulative_emissions[-1,iscen,0,0] - f.cumulative_emissions[first_negative[iscen], iscen, 0, 0])
#f.cumulative_emissions[-2:,iscen,0,0].mean(axis=0) - f.cumulative_emissions[first_negative[iscen], iscen, 0, 0]

In [ ]:
# exclude scenarios that don't reach net zero
netzero_to_2150_cumulative_emissions[netzero_to_2150_cumulative_emissions > 0] = np.nan
netzero_to_2150_cumulative_emissions

In [ ]:
peak_times = np.zeros((97, 1001))
#for iscen in range(97):
#    for iconfig in range(1001):
#        peak_times[iscen, iconfig] = np.array([f.temperature[:,iscen,iconfig,0]]).argmax()
peak_times = f.temperature[:,:,:,0].argmax(axis=0)

In [ ]:
#pl.hist(peak_times[2,:])

In [ ]:
first_negative

In [ ]:
temperature_drawdown = np.zeros((97,1001))
for iscen in tqdm(range(97)):
    for iconfig in range(1001):
        temperature_drawdown[iscen, iconfig] = f.temperature[first_negative[iscen],iscen,iconfig,0]-f.temperature[-1,iscen,iconfig,0]

In [ ]:
pl.hist(temperature_drawdown[2,:])

In [ ]:
pc05 = np.percentile(temperature_drawdown, 5, axis=1)
pc50 = np.percentile(temperature_drawdown, 50, axis=1)
pc95 = np.percentile(temperature_drawdown, 95, axis=1)

netzero = ~np.isnan(netzero_to_2150_cumulative_emissions)

In [ ]:
netzero_to_2150_cumulative_emissions[netzero]

In [ ]:
netzero & (pc05 > 0)

In [ ]:
pl.scatter(-pc50[netzero & (pc50>0)], netzero_to_2150_cumulative_emissions[netzero & (pc50>0)])
pl.scatter(-pc05[netzero & (pc05>0)], netzero_to_2150_cumulative_emissions[netzero & (pc05>0)])
pl.scatter(-pc95[netzero & (pc95>0)], netzero_to_2150_cumulative_emissions[netzero & (pc95>0)])

sl50, ic50, _, _, _ = linregress(-pc50[netzero & (pc50>0)], netzero_to_2150_cumulative_emissions[netzero & (pc50>0)])
x = np.linspace(-0.35, 0)
pl.plot(x, sl50*x + ic50)
print(sl50, ic50)

sl05, ic05, _, _, _ = linregress(-pc05[netzero & (pc05>0)], netzero_to_2150_cumulative_emissions[netzero & (pc05>0)])
x = np.linspace(-0.2, 0)
pl.plot(x, sl05*x + ic05)
print(sl05, ic05)

sl95, ic95, _, _, _ = linregress(-pc95[netzero & (pc95>0)], netzero_to_2150_cumulative_emissions[netzero & (pc95>0)])
x = np.linspace(-0.5, 0)
pl.plot(x, sl95*x + ic95)
print(sl95, ic95)

pl.xlabel('Temperature drawdown, peak to 2150')
pl.ylabel('Cumulative CO2, net zero to 2150, GtCO2')

In [ ]:
#c = sl50 * t + ic50
3664/sl50  # TCRE 

In [ ]:
ic50/sl50  # -ZEC

In [ ]:
3664/sl05, 3664/sl95

In [ ]:
ic05/sl05, ic95/sl95  # take negative of

In [ ]:
netzero_to_2150_cumulative_emissions[~np.isnan(netzero_to_2150_cumulative_emissions)]

In [ ]:
pc05 = np.percentile(temperature_drawdown, 5, axis=1)
pc05

In [ ]:
netzero_to_2150_cumulative_emissions[~np.isnan(netzero_to_2150_cumulative_emissions)&(pc05>0)]

In [ ]:
pl.scatter(-temperature_drawdown[netzero,11], netzero_to_2150_cumulative_emissions[netzero])

In [ ]:
pl.scatter(-temperature_drawdown[netzero,12], netzero_to_2150_cumulative_emissions[netzero])

In [ ]:
pl.plot(f.temperature[:,:,12,0]);

In [ ]:
dataout = np.zeros((1001, 5))

for iconfig, config in enumerate(fair_params_df.index):
    sl, ic, _, _, _ = linregress(-temperature_drawdown[netzero,iconfig], netzero_to_2150_cumulative_emissions[netzero])
    print(sl, ic, 3664/sl, -ic/sl)
    dataout[iconfig, :] = [config, sl, ic, 3664/sl, -ic/sl]

In [ ]:
df = pd.DataFrame(dataout, columns=['config', 'slope', 'intercept', 'tcre_postpeak', 'zec'])

In [ ]:
df = df.astype({'config': 'int32'})

In [ ]:
df.set_index('config', inplace=True)

In [ ]:
df

In [ ]:
df.to_csv('../data/fair_1001_c1_scenarios_tcre_zec_2150.csv')